In [1]:
import pandas as pd
games = pd.read_csv('spreadspoke_scores.csv')
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13232 entries, 0 to 13231
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        13232 non-null  object 
 1   schedule_season      13232 non-null  int64  
 2   schedule_week        13232 non-null  object 
 3   schedule_playoff     13232 non-null  bool   
 4   team_home            13232 non-null  object 
 5   score_home           13231 non-null  float64
 6   score_away           13231 non-null  float64
 7   team_away            13232 non-null  object 
 8   team_favorite_id     10753 non-null  object 
 9   spread_favorite      10753 non-null  float64
 10  over_under_line      10743 non-null  object 
 11  stadium              13232 non-null  object 
 12  stadium_neutral      13232 non-null  bool   
 13  weather_temperature  12189 non-null  float64
 14  weather_wind_mph     12172 non-null  float64
 15  weather_humidity     8441 non-null  

remove data before 1979 because there was no betting data prior

In [2]:
games = games[games['schedule_season'] >= 1979]
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
2501,9/1/1979,1979,1,False,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,TB,-3.0,30,Houlihan's Stadium,False,79.0,9.0,87.0,NaN
2502,9/2/1979,1979,1,False,Buffalo Bills,7.0,9.0,Miami Dolphins,MIA,-5.0,39,Ralph Wilson Stadium,False,74.0,15.0,74.0,NaN
2503,9/2/1979,1979,1,False,Chicago Bears,6.0,3.0,Green Bay Packers,CHI,-3.0,31,Soldier Field,False,78.0,11.0,68.0,NaN
2504,9/2/1979,1979,1,False,Denver Broncos,10.0,0.0,Cincinnati Bengals,DEN,-3.0,31.5,Mile High Stadium,False,69.0,6.0,38.0,NaN
2505,9/2/1979,1979,1,False,Kansas City Chiefs,14.0,0.0,Baltimore Colts,KC,-1.0,37,Arrowhead Stadium,False,76.0,8.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227,1/23/2022,2021,Division,True,Kansas City Chiefs,42.0,36.0,Buffalo Bills,KC,-2.5,54.0,Arrowhead Stadium,False,35.0,6.0,54.0,NaN
13228,1/23/2022,2021,Division,True,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,TB,-3.0,48.0,Raymond James Stadium,False,50.0,11.0,69.0,NaN
13229,1/30/2022,2021,Conference,True,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,KC,-7.0,54.5,Arrowhead Stadium,False,41.0,4.0,41.0,NaN
13230,1/30/2022,2021,Conference,True,Los Angeles Rams,20.0,17.0,San Francisco 49ers,LAR,-3.5,46.0,SoFi Stadium,False,72.0,0.0,NaN,DOME


creating columns for spread covering, total points, o/u covered, etc...

There are missing values for the over/under line through the 1985 season. While I could just remove the missing values, I am going to remove all data 1985 and before because I want to do some analysis over full seasons, so havnig a few missing entries from those seasons would mess that up. 1986 and on is still a considerable sized data frame.

In [3]:
games = games[games['schedule_season'] > 1985]
games['over_under_line'] = games['over_under_line'].astype(float)
games['total_points'] = games['score_home'] + games['score_away']
games.loc[games['total_points'] > games['over_under_line'], 'over_under_result'] = 'over'
games.loc[games['total_points'] < games['over_under_line'], 'over_under_result'] = 'under'
games.loc[games['total_points'] == games['over_under_line'], 'over_under_result'] = 'push'

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be s

In [4]:
games['team_favorite_id'].unique()

array(['NYJ', 'CHI', 'DEN', 'GB', 'CIN', 'MIN', 'NE', 'NO', 'MIA', 'SEA',
       'LAR', 'SF', 'WAS', 'NYG', 'ATL', 'DAL', 'TEN', 'CLE', 'BUF',
       'DET', 'KC', 'LVR', 'LAC', 'PICK', 'TB', 'PHI', 'PIT', 'ARI',
       'IND', 'BAL', 'JAX', 'CAR', 'HOU'], dtype=object)

In [5]:
teams = pd.read_csv('nfl_teams.csv')



team_dict = dict(zip(teams.team_name, teams.team_id))
games['home_id'] = games['team_home'].map(team_dict)
games['away_id'] = games['team_away'].map(team_dict)

games.loc[games['team_favorite_id'] == games['home_id'], 'underdog_team_id'] = games['away_id']
games.loc[games['team_favorite_id'] == games['away_id'], 'underdog_team_id'] = games['home_id']
games.loc[games['team_favorite_id'] == 'PICK', 'underdog_team_id'] = games['team_favorite_id']

games['winning_team'] = 'team'

games.loc[games['score_home'] > games['score_away'], 'winning_team'] = games['home_id']
games.loc[games['score_home'] < games['score_away'], 'winning_team'] = games['away_id']
games.loc[games['score_home'] == games['score_away'], 'winning_team'] = 'tie'

games.loc[games['score_home'] < games['score_away'], 'losing_team'] = games['home_id']
games.loc[games['score_home'] > games['score_away'], 'losing_team'] = games['away_id']
games.loc[games['score_home'] == games['score_away'], 'losing_team'] = 'tie'

games.loc[games['winning_team'] == 'tie', 'tying_team_home'] = games['home_id']
games.loc[games['winning_team'] == 'tie', 'tying_team_away'] = games['away_id']

# games['winning_team'].value_counts() #one entry not changed from 'team': need to look at

#games.loc[games['winning_team'] == 'team']
#data must not have been updated since the Superbowl this year because the scores for it are missing
#will remove that entry

games = games.drop(labels=13231, axis=0)



/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [6]:
games.loc[games['winning_team'] == games['team_favorite_id'], 'fav_or_dog_won'] = 'fav'
games.loc[games['winning_team'] == games['underdog_team_id'], 'fav_or_dog_won'] = 'dog'
games.loc[games['team_favorite_id'] == 'PICK', 'fav_or_dog_won'] = 'PICK'

games['fav_or_dog_won'].value_counts()

fav     6038
dog     3040
PICK      94
Name: fav_or_dog_won, dtype: int64

In [7]:
games['margin_of_victory'] = abs(games['score_home'] - games['score_away'])

games.loc[games['margin_of_victory'] > abs(games['spread_favorite']), 'spread_result'] = 'fav'
games.loc[games['margin_of_victory'] < abs(games['spread_favorite']), 'spread_result'] = 'dog'
games.loc[games['margin_of_victory'] == abs(games['spread_favorite']), 'spread_result'] = 'push'
games.loc[games['winning_team'] == games['underdog_team_id'], 'spread_result'] = 'dog'
games.loc[games['team_favorite_id'] == 'PICK', 'spread_result'] = 'PICK'



games['spread_result'].value_counts()

dog     4564
fav     4281
push     252
PICK      94
Name: spread_result, dtype: int64

In [8]:
games.loc[games['spread_result'] == 'fav', 'spread_winner'] = games['team_favorite_id']
games.loc[games['spread_result'] == 'dog', 'spread_winner'] = games['underdog_team_id']
games.loc[games['spread_result'] == 'fav', 'spread_loser'] = games['underdog_team_id']
games.loc[games['spread_result'] == 'dog', 'spread_loser'] = games['team_favorite_id']
games.loc[games['spread_result'] == 'PICK', 'spread_winner'] = games['winning_team']
games.loc[games['spread_result'] == 'PICK', 'spread_loser'] = games['losing_team']

games.loc[games['spread_result'] == 'push', 'push_team_home'] = games['team_home']
games.loc[games['spread_result'] == 'push', 'push_team_away'] = games['team_away']



In [14]:
games.loc[games['over_under_result'] == 'over', 'over_winner_home'] = games['home_id']
games.loc[games['over_under_result'] == 'over', 'over_winner_away'] = games['away_id']

games.loc[games['over_under_result'] == 'under', 'over_loser_home'] = games['home_id']
games.loc[games['over_under_result'] == 'under', 'over_loser_away'] = games['away_id']

games.loc[games['over_under_result'] == 'push', 'over_push_home'] = games['home_id']
games.loc[games['over_under_result'] == 'push', 'over_push_away'] = games['away_id']


In [15]:
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,spread_winner,spread_loser,push_team_home,push_team_away,over_winner_home,over_winner_away,over_loser_home,over_loser_away,over_push_home,over_push_away
4040,9/7/1986,1986,1,False,Buffalo Bills,24.0,28.0,New York Jets,NYJ,-6.0,...,BUF,NYJ,NaN,NaN,BUF,NYJ,NaN,NaN,NaN,NaN
4041,9/7/1986,1986,1,False,Chicago Bears,41.0,31.0,Cleveland Browns,CHI,-10.5,...,CLE,CHI,NaN,NaN,CHI,CLE,NaN,NaN,NaN,NaN
4042,9/7/1986,1986,1,False,Denver Broncos,38.0,36.0,Los Angeles Raiders,DEN,-3.0,...,LVR,DEN,NaN,NaN,DEN,LVR,NaN,NaN,NaN,NaN
4043,9/7/1986,1986,1,False,Green Bay Packers,3.0,31.0,Houston Oilers,GB,-4.0,...,TEN,GB,NaN,NaN,NaN,NaN,GB,TEN,NaN,NaN
4044,9/7/1986,1986,1,False,Kansas City Chiefs,24.0,14.0,Cincinnati Bengals,CIN,-1.0,...,KC,CIN,NaN,NaN,NaN,NaN,KC,CIN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,1/22/2022,2021,Division,True,Tennessee Titans,16.0,19.0,Cincinnati Bengals,TEN,-4.0,...,CIN,TEN,NaN,NaN,NaN,NaN,TEN,CIN,NaN,NaN
13227,1/23/2022,2021,Division,True,Kansas City Chiefs,42.0,36.0,Buffalo Bills,KC,-2.5,...,KC,BUF,NaN,NaN,KC,BUF,NaN,NaN,NaN,NaN
13228,1/23/2022,2021,Division,True,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,TB,-3.0,...,LAR,TB,NaN,NaN,TB,LAR,NaN,NaN,NaN,NaN
13229,1/30/2022,2021,Conference,True,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,KC,-7.0,...,CIN,KC,NaN,NaN,NaN,NaN,KC,CIN,NaN,NaN


In [16]:
games.to_csv('games.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=edd70093-bbe4-4f95-a99a-f731d544cc85' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>